In [1]:
import os  # Operating system interfaces
import tensorflow as tf                                    # TensorFlow deep learning framework
import matplotlib.pyplot as plt                            # Plotting library
import matplotlib.image as mpimg                           # Image loading and manipulation library
from tensorflow.keras.models import Sequential, Model      # Sequential and Functional API for building models
from tensorflow.keras.optimizers import Adam               # Adam optimizer for model training
from tensorflow.keras.callbacks import EarlyStopping       # Early stopping callback for model training
from tensorflow.keras.regularizers import l1, l2           # L1 and L2 regularization for model regularization
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data augmentation and preprocessing for images
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D, BatchNormalization
# Various types of layers for building neural networks
from tensorflow.keras.applications import DenseNet121, EfficientNetB4, Xception, VGG16, VGG19   # Pre-trained models for transfer learning

In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hafsa20","key":"d4df2e155206082c552005f98d4263aa"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d mhantor/apple-leaf-diseases/

Dataset URL: https://www.kaggle.com/datasets/mhantor/apple-leaf-diseases/versions/
License(s): Apache 2.0
 75% 5.00M/6.70M [00:00<00:00, 38.5MB/s]
100% 6.70M/6.70M [00:00<00:00, 44.9MB/s]


In [15]:
!unzip apple-leaf-diseases.zip -d /content/train

Archive:  apple-leaf-diseases.zip
  inflating: /content/train/Apple_black_rot/Black_rot (1).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (10).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (100).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (101).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (102).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (103).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (104).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (105).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (106).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (107).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (108).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (109).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (11).JPG  
  inflating: /content/train/Apple_black_rot/Black_rot (110).JPG  
  inflating: /content/train/Apple_black_rot/Bl

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os

In [11]:
img_height, img_width = 256, 256  # Maintain original resolution
batch_size = 32
num_classes = 10

In [14]:
tf.keras.preprocessing.image_dataset_from_directory

<function keras.src.utils.image_dataset.image_dataset_from_directory(directory, labels='inferred', label_mode='int', class_names=None, color_mode='rgb', batch_size=32, image_size=(256, 256), shuffle=True, seed=None, validation_split=None, subset=None, interpolation='bilinear', follow_links=False, crop_to_aspect_ratio=False, **kwargs)>

In [16]:
os.listdir()

['.config',
 'kaggle.json',
 'train',
 'apple-leaf-diseases.zip',
 'Apple_cedar_rust',
 'Apple_scab',
 'Apple_black_rot',
 'sample_data']

In [18]:
# Define paths
dataset_dir = '/train'

# Efficient Data Loading
AUTOTUNE = tf.data.experimental.AUTOTUNE

def preprocess(image, label):
    image = tf.image.resize(image, [img_height, img_width])
    image = image / 255.0  # Normalize to [0, 1]
    return image, label


# Create training and validation datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    label_mode='categorical',
    shuffle=True,
    seed=42,
    validation_split=0.2,  # Use 20% of the data for validation
    subset='training'      # This will create the training subset
).map(preprocess).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    batch_size=batch_size,
    image_size=(img_height, img_width),
    label_mode='categorical',
    shuffle=True,
    seed=42,
    validation_split=0.2,  # Use 20% of the data for validation
    subset='validation'    # This will create the validation subset
).map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)


Found 480 files belonging to 3 classes.
Using 384 files for training.
Found 480 files belonging to 3 classes.
Using 96 files for validation.


In [22]:
# Count the number of classes in your dataset
num_classes = len(os.listdir(dataset_dir))


model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    GlobalAveragePooling2D(),

    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 127, 127, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 64)       

In [23]:
# Callbacks
callbacks = [
    EarlyStopping(patience=10, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True, verbose=1)
]


history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/50
11/12 [==========================>...] - ETA: 0s - loss: 1.0867 - accuracy: 0.3295
Epoch 1: val_loss improved from inf to 1.08253, saving model to best_model.keras
12/12 [==============================] - 6s 68ms/step - loss: 1.0866 - accuracy: 0.3281 - val_loss: 1.0825 - val_accuracy: 0.3229
Epoch 2/50
11/12 [==========================>...] - ETA: 0s - loss: 1.0512 - accuracy: 0.4119
Epoch 2: val_loss improved from 1.08253 to 0.99982, saving model to best_model.keras
12/12 [==============================] - 0s 40ms/step - loss: 1.0472 - accuracy: 0.4271 - val_loss: 0.9998 - val_accuracy: 0.3229
Epoch 3/50
11/12 [==========================>...] - ETA: 0s - loss: 0.9202 - accuracy: 0.5227
Epoch 3: val_loss improved from 0.99982 to 0.79968, saving model to best_model.keras
12/12 [==============================] - 0s 40ms/step - loss: 0.9113 - accuracy: 0.5391 - val_loss: 0.7997 - val_accuracy: 0.6979
Epoch 4/50
11/12 [==========================>...] - ETA: 0s - loss: 0.7076 - 